In [114]:
import pandas as pd
import re
import numpy as np

In [129]:
#Open clean data without index number
gta_original = pd.read_csv('./gta_original.csv')

In [130]:
gta_original = gta_original.drop(['Unnamed: 0'],axis=1)

In [131]:
gta_original = gta_original.drop_duplicates(subset= ['Address'])

In [132]:
gta_original = gta_original.reset_index()

In [133]:
gta_original = gta_original.drop(['index'],axis=1)

In [134]:
gta_original.index += 1

In [135]:
gta_original = gta_original.reset_index(level=0)

In [137]:
gta_original = gta_original.rename(columns = {'index': 'Id'})

In [138]:
gta_original

,Id,PublicRemarks,Bedrooms,Building_Type,Address,LeaseRent,Property_Type,OwnershipType,ParkingSpaceTotal
0,1,Excellent Location!Ready To Move In With Other...,1,Other,"#BSMNT -12 FLERIMAC RD|Toronto, Ontario M1E4A9",$700/Monthly,Single Family,NaN,NaN
1,2,Roommate Rents Out Master Bedroom Condo Unit. ...,2,Apartment,"4101 SHEPPARD AVE E|Toronto, Ontario M1S3H3",$800/Monthly,Single Family,Condominium/Strata,NaN
2,3,2 Individual Furnished Bdrms With Private Entr...,1,House,"107 BRAYMORE BLVD|Toronto, Ontario M1B2N6",$900/Monthly,Single Family,Freehold,2.0
3,4,"A Large Den For A Single Lady, Private Washroo...",1 + 1,Apartment,"#906 -150 MAIN ST W|Hamilton, Ontario L8P1H8",$900/Monthly,Single Family,Condominium/Strata,NaN
4,5,Fully Furnished Room Available For Lease In A ...,1,House,"#(ROOM) -1036 MCCUAIG DR|Milton, Ontario L9T6S3",$900/Monthly,Single Family,Freehold,NaN
...,...,...,...,...,...,...,...,...,...
491,492,Freshly Renovated 2 Bed Loft Style Unit With E...,2,Duplex,"68 HIGHLAND RD|Kitchener, Ontario N2M3B5","$1,800/Monthly",Single Family,NaN,NaN
492,493,Welcome to this spacious 1 bedroom plus den lo...,0 + 1,House,"35 VICMOUNT Drive|Kitchener, Ontario N2M2A2","$1,800/Monthly",Single Family,Freehold,NaN
493,494,Checks All Boxes. Spacious 2Bdr + 1Bth Walk-Ou...,2,House,"#BSMT -63 WHEATFIELD CRES|Kitchener, Ontario ...","$1,800/Monthly",Single Family,Freehold,5.0
494,495,Introducing Gorgeous Basement Bungalow In Clar...,2,House,"#BSMT -2632 LUNDENE RD|Mississauga, Ontario L...","$1,800/Monthly",Single Family,Freehold,NaN


# DB Insert Functions

In [139]:
!pip install psycopg2

In [140]:
import psycopg2 as ps

def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host = host_name,
                         database = dbname, 
                         user = username, 
                         password = password,
                         port = port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
    return conn


def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS gta_table (
        Id serial PRIMARY KEY,
        Address VARCHAR(255),
        PublicRemarks TEXT,
        Bedrooms TEXT,
        Building_Type VARCHAR(255),
        LeaseRent VARCHAR(255),
        Property_Type VARCHAR(255),
        OwnershipType VARCHAR(255),
        ParkingSpaceTotal VARCHAR(255)
    )""")
    
    curr.execute(create_table_command) 


#write insert command
def insert_into_table(curr, Address, PublicRemarks, Bedrooms, Building_Type, LeaseRent, Property_Type, OwnershipType, ParkingSpaceTotal):
    insert_into_gta_table = ("""INSERT INTO gta_table (Address, PublicRemarks, Bedrooms, Building_Type, LeaseRent, Property_Type, OwnershipType, ParkingSpaceTotal)
                            VALUES(%s,%s,%s,%s,%s,%s,%s,%s);""")

    row_to_insert = (Address, PublicRemarks, Bedrooms, Building_Type, LeaseRent, Property_Type, OwnershipType, ParkingSpaceTotal)
    curr.execute(insert_into_gta_table, row_to_insert) #pass on sql cmd, parameter

    
def append_from_df_to_db(curr, df):
    for i,row in df.iterrows():
        insert_into_table(curr, row['Address'], row['PublicRemarks'], row['Bedrooms'], row['Building_Type'], row['LeaseRent'], row['Property_Type'], row['OwnershipType'], row['ParkingSpaceTotal'])

        


# Execute

In [176]:
host_name = 'localhost'
dbname = 'projects'
port = 5432
username = 'postgres'
password = 'fhfflvhffl'
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)

Connected!


In [177]:
curr = conn.cursor()  

In [178]:
curr.execute("""DROP TABLE IF EXISTS gta_table;""")

In [179]:
create_table(curr) 

In [180]:
conn.commit()

In [181]:
#insert new rows into table
append_from_df_to_db(curr,gta_original)  

In [182]:
#commit all the sql code to the database
conn.commit() 